# Datafiles in the NWB format

We use the NWB data standard to store and share our multimodal neurophysiological recordings:

From the NWB  webiste https://www.nwb.org/:

> Neurodata Without Borders: Neurophysiology (NWB:N) is a data standard for neurophysiology, providing neuroscientists with a common standard to share, archive, use, and build analysis tools for neurophysiology data. NWB:N is designed to store a variety of neurophysiology data, including data from intracellular and extracellular electrophysiology experiments, data from optical physiology experiments, and tracking and stimulus data.

We rely on the python API to create such files, see:

https://pynwb.readthedocs.io/en/stable/

The script to build such data files is [build_NWB.py](./build_NWB.py) and we update those files (e.g. to add processed data) the script [update_NWB.py](./update_NWB.py)



In [2]:
# lading the python NWB API
import pynwb
import numpy as np # numpy for numerical analysis
import matplotlib.pylab as plt # + matplotlib for vizualization

In [4]:
# loading an example file
filename = '/media/yann/Yann/2020_11_12/17-54-24/2020_11_12-17-54-24.FULL.nwb'
io = pynwb.NWBHDF5IO(filename, 'r')
nwbfile = io.read() # don't forget to close afterwards !! (io.close() )

In [6]:
# let's have a look at what is inside
print(nwbfile)

root pynwb.file.NWBFile at 0x139903994860624
Fields:
  acquisition: {
    FaceCamera-acquisition <class 'pynwb.image.ImageSeries'>,
    Photodiode-Signal <class 'pynwb.base.TimeSeries'>,
    Running-Speed <class 'pynwb.base.TimeSeries'>
  }
  experiment_description: full-field-drifting-gratings-5-orientations-10-repeats
  experimenter: ['Yann Zerlaut']
  file_create_date: [datetime.datetime(2021, 1, 5, 10, 10, 23, 343783, tzinfo=tzoffset(None, 3600))]
  identifier: 2020_11_12-17-54-24
  institution: Institut du Cerveau et de la Moelle, Paris
  lab: Rebola and Bacci labs
  processing: {
    ophys <class 'pynwb.base.ProcessingModule'>
  }
  session_description: {'NIdaq-acquisition-frequency': 10000, 'NIdaq-analog-input-channels': 1, 'NIdaq-digital-input-channels': 2, 'protocol_folder': 'exp\\protocols', 'root_datafolder': 'DATA', 'FaceCamera-frame-rate': 20, 'VisualStim': True, 'Locomotion': True, 'Electrophy': False, 'FaceCamera': True, 'CaImaging': True, 'protocol': 'full-field-driftin

## Acquisition fields

In [5]:
nwbfile.acquisition

{'FaceCamera-acquisition': FaceCamera-acquisition pynwb.image.ImageSeries at 0x139904004361296
 Fields:
   comments: no comments
   conversion: 1.0
   data: <HDF5 dataset "data": shape (7073, 680, 1280), type "|u1">
   description: no description
   interval: 1
   resolution: -1.0
   timestamps: <HDF5 dataset "timestamps": shape (7073,), type "<f8">
   timestamps_unit: seconds
   unit: NA,
 'Photodiode-Signal': Photodiode-Signal pynwb.base.TimeSeries at 0x139903995178256
 Fields:
   comments: no comments
   conversion: 1.0
   data: <HDF5 dataset "data": shape (3600000,), type "<f8">
   description: no description
   rate: 10000.0
   resolution: -1.0
   starting_time: 0.0
   starting_time_unit: seconds
   unit: [current],
 'Running-Speed': Running-Speed pynwb.base.TimeSeries at 0x139903994714448
 Fields:
   comments: no comments
   conversion: 1.0
   data: <HDF5 dataset "data": shape (3595000,), type "<f8">
   description: no description
   rate: 10000.0
   resolution: -1.0
   starting_

In [9]:
nwbfile.acquisition['FaceCamera-acquisition']._AbstractContainer__container_source

'/media/yann/Yann/2020_11_12/17-54-24/2020_11_12-17-54-24.FULL.nwb'

## Processing fields

In [11]:
io.close()